# 라쏘(alpha=0.01)로 선택한 변수로 돌린 모델(ver5.0) 변수 23개

'장애등급 _4', '가구 유형 _21', '가장 불편한부위_36', '질병명...38_171', '주된 진단명...40_6',
       '현재 지속적 진료여부_1', '현재 지속적 진료여부_2', '인플루엔자 예방접종여부_1', '인플루엔자 예방접종여부_2',
       '혈당관리 치료 여부_0', '지난 2주간 아팠던 날수_14', '본인 체형 평가_4', '2)이용_만성질환관리_1',
       '3)이용_장애관리_재활_2', '지팡이-필요_1', '지팡이-필요_2', '지팡이-소지_1', '지팡이-소지_2',
       '지팡이-사용_0', '휴대폰 사용 여부_1', '스마트폰 사용 여부_1', '검퓨터 사용 여부_1',
       '검퓨터 사용 여부_2'

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
import pickle
import os
from sklearn.model_selection import train_test_split

In [5]:
df=pd.read_excel("지체장애_고혈압_결측치 대체_데이터.xlsx")
df.head()

,id,생년,장애등록 연도,본인을 포함한 총 가구원수,본인을 포함한 총 장애인수,월 평균 총가구소득,가구 주된 수입원,가구 월평균 지출액,장애발생시 연령_36,월 혈압약 일수,...,인공호흡기-소지_673,인공호흡기-사용_674,사용빈도,1일 이용시간,이용경험-상담과 평가,휴대폰 사용 여부,스마트폰 사용 여부,컴퓨터 사용 여부,인터넷 사용 여부,01)만성질환명(고혈압)
0,1,1970,2008,3,1,280,1,250,38,0,...,2,0,0,0,2,2,1,1,1,0
1,2,1949,2010,3,1,180,3,150,20,30,...,2,0,0,0,2,1,2,2,2,1
2,4,1958,2011,2,1,350,1,200,51,0,...,2,0,0,0,2,2,1,2,2,0
3,5,1955,2009,3,2,154,1,154,53,30,...,2,0,1,3,2,2,1,2,2,1
4,6,1936,2010,2,1,275,3,270,62,0,...,2,0,2,9,1,1,2,2,2,0


In [4]:
df.columns

Index(['id', '생년', '장애등록 연도', '본인을 포함한 총 가구원수', '본인을 포함한 총 장애인수', '월 평균 총가구소득',
       '가구 주된 수입원', '가구 월평균 지출액', '장애발생시 연령_36', '월 혈압약 일수',
       ...
       '인공호흡기-소지_673', '인공호흡기-사용_674', '사용빈도', '1일 이용시간', '이용경험-상담과 평가',
       '휴대폰 사용 여부', '스마트폰 사용 여부', '컴퓨터 사용 여부', '인터넷 사용 여부', '01)만성질환명(고혈압)'],
      dtype='object', length=267)

In [6]:
df1=df[['장애등급', '가구 유형', '가장 불편한부위', '질병명_38', '주된 진단명_40',
       '현재 지속적 진료여부',  '인플루엔자 예방접종여부', 
       '혈당관리 치료 여부', '지난 2주간 아팠던 날수', '본인 체형 평가', '2)이용_만성질환관리',
       '3)이용_장애관리_재활', '지팡이-필요', '지팡이-소지', 
       '지팡이-사용', '휴대폰 사용 여부', '스마트폰 사용 여부', '컴퓨터 사용 여부','01)만성질환명(고혈압)']]
df1 #2686건

,장애등급,가구 유형,가장 불편한부위,질병명_38,주된 진단명_40,현재 지속적 진료여부,인플루엔자 예방접종여부,혈당관리 치료 여부,지난 2주간 아팠던 날수,본인 체형 평가,2)이용_만성질환관리,3)이용_장애관리_재활,지팡이-필요,지팡이-소지,지팡이-사용,휴대폰 사용 여부,스마트폰 사용 여부,컴퓨터 사용 여부,01)만성질환명(고혈압)
0,6,21,36,0,12,1,2,0,14,3,1,2,2,2,0,2,1,1,0
1,6,21,14,0,8,1,1,0,14,2,1,1,1,2,0,1,2,2,1
2,5,11,38,175,11,1,2,0,14,3,1,1,2,2,0,2,1,2,0
3,5,29,38,175,11,1,1,0,14,3,1,1,1,1,1,2,1,2,1
4,3,24,36,171,6,1,1,1,14,3,1,1,2,2,0,1,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,3,11,36,0,12,1,1,0,14,3,1,2,1,1,1,1,2,2,0
2682,5,11,38,0,12,1,1,0,0,4,1,1,2,2,0,2,1,1,0
2683,6,11,38,175,11,1,2,0,14,4,1,1,2,2,0,2,1,2,1
2684,6,11,31,175,11,1,2,0,14,3,1,2,2,2,0,2,1,2,0


### 결측치가 없는 관측치 하나 미리 빼놓기 (테스트용)

In [7]:
df1[df1['01)만성질환명(고혈압)']==1]

,장애등급,가구 유형,가장 불편한부위,질병명_38,주된 진단명_40,현재 지속적 진료여부,인플루엔자 예방접종여부,혈당관리 치료 여부,지난 2주간 아팠던 날수,본인 체형 평가,2)이용_만성질환관리,3)이용_장애관리_재활,지팡이-필요,지팡이-소지,지팡이-사용,휴대폰 사용 여부,스마트폰 사용 여부,컴퓨터 사용 여부,01)만성질환명(고혈압)
1,6,21,14,0,8,1,1,0,14,2,1,1,1,2,0,1,2,2,1
3,5,29,38,175,11,1,1,0,14,3,1,1,1,1,1,2,1,2,1
5,5,11,38,0,12,1,1,0,14,4,1,1,1,1,1,1,2,2,1
6,3,32,14,0,8,1,1,0,14,2,1,1,1,1,1,1,2,2,1
7,5,11,36,171,6,1,1,0,14,4,1,1,1,1,2,2,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2678,5,11,36,171,6,1,1,0,14,2,1,1,2,2,0,2,1,1,1
2679,6,11,38,0,12,1,1,0,14,1,1,1,1,1,1,2,2,2,1
2680,2,11,45,0,6,1,1,1,14,4,1,1,1,1,1,1,2,2,1
2683,6,11,38,175,11,1,2,0,14,4,1,1,2,2,0,2,1,2,1


In [68]:
out=df1.loc[[3]] # 고혈압 3, 32유 , 46,50무  
out

,장애등급,가구 유형,가장 불편한부위,질병명_38,주된 진단명_40,현재 지속적 진료여부,인플루엔자 예방접종여부,혈당관리 치료 여부,지난 2주간 아팠던 날수,본인 체형 평가,2)이용_만성질환관리,3)이용_장애관리_재활,지팡이-필요,지팡이-소지,지팡이-사용,휴대폰 사용 여부,스마트폰 사용 여부,컴퓨터 사용 여부,01)만성질환명(고혈압)
3,5,29,38,175,11,1,1,0,14,3,1,1,1,1,1,2,1,2,1


In [69]:
df2=df1.drop(out.index).reset_index(drop=True)
df2.head()

,장애등급,가구 유형,가장 불편한부위,질병명_38,주된 진단명_40,현재 지속적 진료여부,인플루엔자 예방접종여부,혈당관리 치료 여부,지난 2주간 아팠던 날수,본인 체형 평가,2)이용_만성질환관리,3)이용_장애관리_재활,지팡이-필요,지팡이-소지,지팡이-사용,휴대폰 사용 여부,스마트폰 사용 여부,컴퓨터 사용 여부,01)만성질환명(고혈압)
0,6,21,36,0,12,1,2,0,14,3,1,2,2,2,0,2,1,1,0
1,6,21,14,0,8,1,1,0,14,2,1,1,1,2,0,1,2,2,1
2,5,11,38,175,11,1,2,0,14,3,1,1,2,2,0,2,1,2,0
3,3,24,36,171,6,1,1,1,14,3,1,1,2,2,0,1,2,2,0
4,5,11,38,0,12,1,1,0,14,4,1,1,1,1,1,1,2,2,1


In [70]:
y_data=df2[['01)만성질환명(고혈압)']]

In [81]:
x_data=df2.drop(['01)만성질환명(고혈압)'],axis=1)
x_data

,장애등급,가구 유형,가장 불편한부위,질병명_38,주된 진단명_40,현재 지속적 진료여부,인플루엔자 예방접종여부,혈당관리 치료 여부,지난 2주간 아팠던 날수,본인 체형 평가,2)이용_만성질환관리,3)이용_장애관리_재활,지팡이-필요,지팡이-소지,지팡이-사용,휴대폰 사용 여부,스마트폰 사용 여부,컴퓨터 사용 여부
0,6,21,36,0,12,1,2,0,14,3,1,2,2,2,0,2,1,1
1,6,21,14,0,8,1,1,0,14,2,1,1,1,2,0,1,2,2
2,5,11,38,175,11,1,2,0,14,3,1,1,2,2,0,2,1,2
3,3,24,36,171,6,1,1,1,14,3,1,1,2,2,0,1,2,2
4,5,11,38,0,12,1,1,0,14,4,1,1,1,1,1,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2680,3,11,36,0,12,1,1,0,14,3,1,2,1,1,1,1,2,2
2681,5,11,38,0,12,1,1,0,0,4,1,1,2,2,0,2,1,1
2682,6,11,38,175,11,1,2,0,14,4,1,1,2,2,0,2,1,2
2683,6,11,31,175,11,1,2,0,14,3,1,2,2,2,0,2,1,2


In [82]:
y_data.value_counts()

01)만성질환명(고혈압)
1                1527
0                1158
dtype: int64

In [83]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=777)

In [84]:
transformer = make_column_transformer(
        (OneHotEncoder(handle_unknown = "ignore"), ['장애등급', '가구 유형', '가장 불편한부위', '질병명_38', '주된 진단명_40',
       '현재 지속적 진료여부',  '인플루엔자 예방접종여부', 
       '혈당관리 치료 여부', '지난 2주간 아팠던 날수', '본인 체형 평가', '2)이용_만성질환관리',
       '3)이용_장애관리_재활', '지팡이-필요', '지팡이-소지', 
       '지팡이-사용', '휴대폰 사용 여부', '스마트폰 사용 여부', '컴퓨터 사용 여부']),
        remainder='passthrough')
transformer = make_pipeline(transformer, MaxAbsScaler())
transformer.fit(x_train)
x_train = transformer.transform(x_train)
x_test = transformer.transform(x_test)

In [85]:
if not os.path.exists('models/disabled_chronic_disease/'):
    os.makedirs('models/disabled_chronic_disease/')

with open('models/disabled_chronic_disease/transformer.pkl', 'wb') as f:
    pickle.dump(transformer, f)

In [96]:
#모델 생성

model = SVC(kernel='rbf', C=5.0, gamma=0.01, probability=True)#,class_weight=class_weight)

In [97]:
#모델 학습
model.fit(x_train, y_train)

#모델 검증
print('training data fit :', model.score(x_train, y_train)) 

print('test data fit :', model.score(x_test, y_test)) 

with open('models/disabled_chronic_disease/transformer.pkl', 'wb') as f:
    pickle.dump(model, f)

C:\Users\LJM\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


training data fit : 0.7062267163384779
test data fit : 0.674937965260546


In [80]:
x_test=out.drop(['01)만성질환명(고혈압)'],axis=1)
x_test

,장애등급,가구 유형,가장 불편한부위,질병명_38,주된 진단명_40,현재 지속적 진료여부,인플루엔자 예방접종여부,혈당관리 치료 여부,지난 2주간 아팠던 날수,본인 체형 평가,2)이용_만성질환관리,3)이용_장애관리_재활,지팡이-필요,지팡이-소지,지팡이-사용,휴대폰 사용 여부,스마트폰 사용 여부,컴퓨터 사용 여부
3,5,29,38,175,11,1,1,0,14,3,1,1,1,1,1,2,1,2


In [ ]:
labels = ['무', '유']

In [ ]:
x_test = transformer.transform(x_test)

y_predict = model.predict(x_test)
label = labels[y_predict[0]]
y_predict = model.predict_proba(x_test)
confidence = y_predict[0][y_predict[0].argmax()]

print(label, confidence) 

유 0.5598681087375312


---